<a href="https://colab.research.google.com/github/Translator-CATRAX/useful_tools/blob/main/run_test_suite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ENTER TEST SUITE NAME AND ENVIRONMENT
test_suite_name = 'sprint_6_tests.json'
url = 'https://arax.ci.transltr.io/api/arax/v1.4/query'
affects_query = {
    "message": {
        "query_graph": {
            "nodes": {
                "ON": {
                    "ids": [],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
                "SN": {
                    "ids": [],
                    "categories": [
                        "biolink:ChemicalEntity"
                    ]
                }
            },
            "edges": {
                "t_edge": {
                    "object": "ON",
                    "subject": "SN",
                    "predicates": [
                        "biolink:affects"
                    ],
                    "knowledge_type": "inferred",
                    "qualifier_constraints": [
                        {
                            "qualifier_set": [
                                {
                                    "qualifier_type_id": "biolink:object_aspect_qualifier",
                                    "qualifier_value": ""
                                },
                                {
                                    "qualifier_type_id": "biolink:object_direction_qualifier",
                                    "qualifier_value": ""
                                }
                            ]
                        }
                    ]
                }
            }
        }
    }
}


treats_query = {
  "message": {
    "query_graph": {
      "nodes": {
        "ON": {
          "ids": [],
          "categories": [
            "biolink:Disease"
          ]
        },
        "SN": {
          "categories": [
            "biolink:ChemicalEntity"
          ]
        }
      },
      "edges": {
        "e01": {
          "object": "ON",
          "subject": "SN",
          "predicates": [
            "biolink:treats"
          ],
          "knowledge_type": "inferred"
        }
      }
    }
  }
}

In [ ]:
def get_safe(element, *keys):
    """
    :param element: JSON to be processed
    :param keys: list of keys in order to be traversed. e.g. "fields","data","message","results
    :return: the value of the terminal key if present or None if not
    """
    if element is None:
        return None
    _element = element
    for key in keys:
        try:
            _element = _element[key]
            if _element is None:
                return None
            if key == keys[-1]:
                return _element
        except KeyError:
            return None
    return None
def generate_message(predicate, creative, input_curie,biolink_object_aspect_qualifier,biolink_object_direction_qualifier,input_category):
    """Create a message to send to Translator services"""

    if predicate == 'treats':
        template = treats_query
    else:
        template = affects_query
    query = deepcopy(template)
    nodes = get_safe(query, "message", "query_graph", "nodes")
    edges = get_safe(query, "message", "query_graph", "edges")
    if biolink_object_aspect_qualifier == '' and biolink_object_direction_qualifier == '' and input_category == 'biolink:Disease':
        for node_val in nodes.values():
            if 'ids' in node_val:
                node_val['ids'].append(input_curie)
    else:
        if input_category == 'biolink:Gene':
            nodes['ON']['ids'].append(input_curie)
            del nodes['SN']['ids']
            edges['t_edge']['qualifier_constraints'][0]['qualifier_set'][0]['qualifier_value'] = biolink_object_aspect_qualifier
            edges['t_edge']['qualifier_constraints'][0]['qualifier_set'][1]['qualifier_value'] = biolink_object_direction_qualifier

        elif input_category == 'biolink:ChemicalEntity':
            nodes['SN']['ids'].append(input_curie)
            del nodes['ON']['ids']
            edges['t_edge']['qualifier_constraints'][0]['qualifier_set'][0]['qualifier_value'] = biolink_object_aspect_qualifier
            edges['t_edge']['qualifier_constraints'][0]['qualifier_set'][1]['qualifier_value'] = biolink_object_direction_qualifier
        else:
            query = {"error": f"unsupported input category provided: {input_category}"}
    return query

In [ ]:
import os
import subprocess

def clone_or_pull(repo_url, target_dir):
    """
    Clones a repository if it doesn't exist, or pulls the latest changes if it does.

    :param repo_url: URL of the Git repository
    :param target_dir: Directory where the repository should be cloned
    """
    if os.path.isdir(os.path.join(target_dir, ".git")):
        print("Repository already cloned. Pulling latest changes...")
        subprocess.run(["git", "-C", target_dir, "pull"], check=True)
    else:
        print("Repository not found. Cloning...")
        subprocess.run(["git", "clone", repo_url, target_dir], check=True)

repo_url = "https://github.com/NCATSTranslator/Tests.git"
target_dir = "Tests"

# Call the function
clone_or_pull(repo_url, target_dir)


Repository already cloned. Pulling latest changes...


In [ ]:
test_suite_name

'sprint_6_tests.json'

In [ ]:
import json
from copy import deepcopy
import requests
import json
!pip install ARS_Test_Runner
from ARS_Test_Runner.semantic_test import pass_fail_analysis
import asyncio
!pip install nest_asyncio

import nest_asyncio
nest_asyncio.apply()
agent = 'arax'
def async_to_sync(awaitable):
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(awaitable)

final_results = []
with open(f'Tests/test_suites/{test_suite_name}') as fp:
    test_suite = json.loads(fp.read())

for test_case, test_case_data in test_suite['test_cases'].items():
    print(test_case)
    predicate = test_case_data['test_case_predicate_name']
    creative = True
    input_curie = test_case_data['test_case_input_id']
    input_category = test_case_data['input_category']
    if input_category is None:
        input_category = test_case_data['test_assets'][0]["input_category"]
    for qualifier in test_case_data['qualifiers']:
        if qualifier['parameter'] == 'biolink_object_aspect_qualifier':
            biolink_object_aspect_qualifier = qualifier['value']
        elif qualifier['parameter'] == 'biolink_object_direction_qualifier':
            biolink_object_direction_qualifier = qualifier['value']

    query = generate_message(predicate, creative, input_curie,biolink_object_aspect_qualifier, biolink_object_direction_qualifier, input_category)
    response = requests.post(url, json=query)
    for asset in test_case_data['test_assets']:
        if response.status_code == 200:
            #Set all assets to FAIL
            json_response = dict(response.json())
            report = {"arax":{}}

            body ={
                      "terms": [
                          asset['output_id']
                      ],
                      "max_synonyms": 10,
                      "format": "slim"
                  }
            node_synonymizer_response = requests.post("https://arax.ncats.io/api/arax/v1.4/entity",json=body)
            node_synonymizer_json_response = dict(node_synonymizer_response.json())
            try:
                output_id = node_synonymizer_json_response[asset['output_id']]["id"]["identifier"]
            except:
                output_id = asset['output_id']

            report = async_to_sync(pass_fail_analysis(report, agent, json_response['message']['results'], output_id, asset['expected_output']))
            result_url = f"https://arax.ci.transltr.io?r={json_response['id'].split('/')[-1]}"

            try:
                if len(json_response['message']['results']) == 0:
                    status = "No results"
                else:
                    status = report[agent]["status"]
            except:
                status = "ERROR"
        else:
            status = "ERROR"
            result_url = ""

        test_result = {
                          "name": asset['name'],
                          "url": result_url,
                          "pk": "-",
                          "TestCase": test_case,
                          "TestAsset": asset['id'],
                          "ars": "-",
                          "aragorn": "-",
                          "arax": status,
                          "bte": "-",
                          "improving": "-",
                          "unsecret": "-",
                          "cqs": "-"
                        }
        final_results.append(test_result)


TestCase_0
TestCase_1
TestCase_2
TestCase_3
TestCase_4
TestCase_5
TestCase_6
TestCase_7
TestCase_8
TestCase_9
TestCase_10
TestCase_11
TestCase_12
TestCase_13
TestCase_14
TestCase_15
TestCase_16
TestCase_17
TestCase_18
TestCase_19
TestCase_20
TestCase_21
TestCase_22
TestCase_23
TestCase_24
TestCase_25
TestCase_26
TestCase_27
TestCase_28
TestCase_29
TestCase_30
TestCase_31
TestCase_32
TestCase_33
TestCase_34
TestCase_35
TestCase_36
TestCase_37
TestCase_38
TestCase_39
TestCase_40
TestCase_41
TestCase_42
TestCase_43
TestCase_44
TestCase_45
TestCase_46
TestCase_47
TestCase_48
TestCase_49
TestCase_50
TestCase_51
TestCase_52
TestCase_53
TestCase_54
TestCase_55
TestCase_56
TestCase_57
TestCase_58
TestCase_59
TestCase_60
TestCase_61
TestCase_62
TestCase_63
TestCase_64
TestCase_65
TestCase_66
TestCase_67
TestCase_68
TestCase_69
TestCase_70
TestCase_71
TestCase_72
TestCase_73
TestCase_74
TestCase_75
TestCase_76
TestCase_77
TestCase_78
TestCase_79
TestCase_80
TestCase_81
TestCase_82
TestCase_83
Te

In [ ]:
len(json_response['message']['results'])

12

In [ ]:
never_show_pass_count = 0
never_show_total_count = 0
acceptable_pass_count = 0
acceptable_total_count = 0
top_answer_pass_count = 0
top_answer_total_count = 0
bad_forgivable_pass_count = 0
bad_forgivable_total_count = 0

for item in final_results:
    if "NeverShow" in item["name"]:
        never_show_total_count += 1
        if item["arax"] == "PASSED":
            never_show_pass_count += 1
    elif "Acceptable" in item["name"]:
        acceptable_total_count += 1
        if item["arax"] == "PASSED":
            acceptable_pass_count += 1
    elif "TopAnswer" in item["name"]:
        top_answer_total_count += 1
        if item["arax"] == "PASSED":
            top_answer_pass_count += 1
    elif "BadButForgivable" in item["name"]:
        bad_forgivable_total_count += 1
        if item["arax"] == "PASSED":
            bad_forgivable_pass_count += 1


print(f"Total Never Show Test Cases  : {never_show_total_count}")
print(f"Passed Never Show Test Cases : {never_show_pass_count}")

print(f"Total Acceptable Test Cases  : {acceptable_total_count}")
print(f"Passed Acceptable Test Cases : {acceptable_pass_count}")

print(f"Total TopAnswer Test Cases  : {top_answer_total_count}")
print(f"Passed TopAnswer Test Cases : {top_answer_pass_count}")

print(f"Total BadButForgivable Test Cases  : {bad_forgivable_total_count}")
print(f"Passed BadButForgivable Test Cases : {bad_forgivable_pass_count}")

print(f"Total Test Cases : {never_show_total_count + acceptable_total_count + top_answer_total_count + bad_forgivable_total_count}")
print(f"Total Passed Cases : {never_show_pass_count + acceptable_pass_count + top_answer_pass_count + bad_forgivable_pass_count}")
print(f"Pass Percentage : {100 * (never_show_pass_count + acceptable_pass_count + top_answer_pass_count + bad_forgivable_pass_count)/(never_show_total_count + acceptable_total_count + top_answer_total_count + bad_forgivable_total_count)}")

Total Never Show Test Cases  : 435
Passed Never Show Test Cases : 410
Total Acceptable Test Cases  : 100
Passed Acceptable Test Cases : 36
Total TopAnswer Test Cases  : 109
Passed TopAnswer Test Cases : 76
Total BadButForgivable Test Cases  : 17
Passed BadButForgivable Test Cases : 13
Total Test Cases : 661
Total Passed Cases : 535
Pass Percentage : 80.93797276853253


In [ ]:
for item in final_results:
    if "Acceptable" in item["name"]:
        print(item)

{'name': 'Acceptable: Ridaforolimus treats Lymphangioleiomyomatosis', 'url': 'https://arax.ci.transltr.io?r=323499', 'pk': '-', 'TestCase': 'TestCase_0', 'TestAsset': 'Asset_328', 'ars': '-', 'aragorn': '-', 'arax': 'FAILED', 'bte': '-', 'improving': '-', 'unsecret': '-', 'cqs': '-'}
{'name': 'Acceptable: L-Glutamine decreases activity or abundance of HBB', 'url': 'https://arax.ci.transltr.io?r=323504', 'pk': '-', 'TestCase': 'TestCase_5', 'TestAsset': 'Asset_398', 'ars': '-', 'aragorn': '-', 'arax': 'FAILED', 'bte': '-', 'improving': '-', 'unsecret': '-', 'cqs': '-'}
{'name': 'Acceptable: Hydroxyurea decreases activity or abundance of HBB', 'url': 'https://arax.ci.transltr.io?r=323504', 'pk': '-', 'TestCase': 'TestCase_5', 'TestAsset': 'Asset_393', 'ars': '-', 'aragorn': '-', 'arax': 'PASSED', 'bte': '-', 'improving': '-', 'unsecret': '-', 'cqs': '-'}
{'name': 'Acceptable: Anticholinergic agents treats Multiple Sclerosis', 'url': 'https://arax.ci.transltr.io?r=323505', 'pk': '-', 'Tes

In [ ]:
import json
file_name = f"final_results_{test_suite_name.split('.')[0]}.json"
ans = {       "row_data": final_results,
              "ara_list": ["arax"]
}
with open(file_name, 'w') as fp:
    json.dump(ans, fp,indent=4)


TestCase_49 Asset_41 PASSED
TestCase_49 Asset_36 FAILED
TestCase_49 Asset_37 PASSED
TestCase_49 Asset_40 PASSED
TestCase_49 Asset_38 PASSED
TestCase_49 Asset_43 PASSED
TestCase_49 Asset_34 PASSED
TestCase_49 Asset_35 PASSED
TestCase_49 Asset_42 PASSED
TestCase_49 Asset_39 PASSED
TestCase_49 Asset_32 FAILED
TestCase_49 Asset_45 PASSED
TestCase_49 Asset_44 PASSED
TestCase_49 Asset_33 FAILED
